**Navigation**
1. [Dependencies and Data Loading](#Dependencies-and-Data-Loading)
2. [Class Weighting](#Class-Weighting)
3. [Fine-Tuning](#Fine-Tuning)
4. [Evaluation](#Evaluation)
5. [Submission Score](#Submission-Score)

# **Dependencies and Data Loading**
Install and import all required libraries for the analysis.

In [ ]:
!pip install pandas datasets transformers torch scikit-learn

import pandas as pd
import os
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch
from tqdm.auto import tqdm
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import BCEWithLogitsLoss
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

tqdm.pandas()

# dataset from hugging face
langs = ['java', 'python', 'pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = load_dataset('NLBSE/nlbse25-code-comment-classification')
ds

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

java_train-00000-of-00001.parquet:   0%|          | 0.00/680k [00:00<?, ?B/s]

java_test-00000-of-00001.parquet:   0%|          | 0.00/174k [00:00<?, ?B/s]

...

Generating python_train split:   0%|          | 0/1884 [00:00<?, ? examples/s]

Generating python_test split:   0%|          | 0/406 [00:00<?, ? examples/s]

Generating pharo_train split:   0%|          | 0/1298 [00:00<?, ? examples/s]

Generating pharo_test split:   0%|          | 0/289 [00:00<?, ? examples/s]

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

## **Class Weighting**


In [ ]:
def compute_class_weights(labels):
    labels = torch.tensor(labels)
    num_samples = labels.shape[0]

    class_counts = labels.sum(axis=0)
    class_weights = num_samples / (len(class_counts) * class_counts)
    return torch.tensor(class_weights).float()

class_weights_dict = {
    lang: compute_class_weights(ds[lang + "_train"]["labels"]).tolist()
    for lang in langs
}

for lang, weights in class_weights_dict.items():
    print(f"Class weights for {lang}: {weights}")

Class weights for java: [0.301305890083313, 4.073836326599121, 2.136963129043579, 0.5196914672851562, 1.2032238245010376, 9.296703338623047, 3.49747371673584]
Class weights for python: [0.6519031524658203, 0.658741295337677, 1.7942856550216675, 1.0985422134399414, 1.0858789682388306]
Class weights for pharo: [1.0417335033416748, 0.3389919102191925, 0.7568512558937073, 4.031055927276611, 1.228003740310669, 0.8664886951446533, 2.439849615097046]


<ipython-input-2-f277a2fef024>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(class_weights).float()


# **Fine-Tuning**

In [ ]:
from transformers import set_seed
import torch.nn as nn

tokenizer = AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")

def preprocess_data(examples):
    return tokenizer(examples['combo'], truncation=True, padding='longest')

for lang in langs:
    ds[lang + '_train'] = ds[lang + '_train'].map(preprocess_data, batched=True)

class MultiLabelClassificationModel(torch.nn.Module):
    def __init__(self, model_name, problem_type, num_labels, class_weights=None):
        super(MultiLabelClassificationModel, self).__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name, problem_type=problem_type, num_labels=num_labels
        )
        self.class_weights = class_weights

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        if labels is not None:
            loss_fn = nn.BCEWithLogitsLoss(pos_weight=self.class_weights.to(logits.device))
            loss = loss_fn(logits, labels.float())
            return {'loss': loss, 'logits': logits}
        return {'logits': logits}

shared_hyperparams = {
    "batch_size": 4,
    "seed": 42,
    "epochs": {
        "java": 6,
        "python": 11,
        "pharo": 12
    }
}

for lang in langs:
    print(f"Length of {lang} train dataset: {len(ds[lang + '_train'])}")
    print(f"Length of {lang} test dataset: {len(ds[lang + '_test'])}")

    set_seed(shared_hyperparams["seed"])

    train_dataset = ds[lang + '_train']

    class_weights = torch.tensor(class_weights_dict[lang], dtype=torch.float32)
    print(f"Class weights for {lang}: {class_weights}")

    num_labels = len(labels[lang])
    model = MultiLabelClassificationModel(
        "microsoft/graphcodebert-base",
        problem_type="multi_label_classification",
        num_labels=num_labels,
        class_weights=class_weights
    )

    if torch.cuda.is_available():
        model = model.to('cuda')

    training_args = TrainingArguments(
        output_dir=f"./graphcodebert_models_{lang}",
        eval_strategy="no",  # Disable validation
        save_strategy="epoch",
        logging_dir=f"./logs_{lang}",
        logging_steps=50,
        num_train_epochs=shared_hyperparams["epochs"][lang],
        per_device_train_batch_size=shared_hyperparams["batch_size"],
        gradient_accumulation_steps=2,
        fp16=True,
        save_total_limit=1,
        load_best_model_at_end=False,
        dataloader_pin_memory=True,
        disable_tqdm=True,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,  # Entire dataset for training
        tokenizer=tokenizer,
    )

    print(f"Training model for {lang}...")
    trainer.train()

    model.model.save_pretrained(f"./graphcodebert_models/{lang}_model")
    tokenizer.save_pretrained(f"./graphcodebert_models/{lang}_tokenizer")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("GraphCodeBERT Training complete for all languages!")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

...

Training model for java...
{'loss': 0.3961, 'grad_norm': 1.605301856994629, 'learning_rate': 4.956232492997199e-05, 'epoch': 0.052521008403361345}
{'loss': 0.2798, 'grad_norm': 1.4618233442306519, 'learning_rate': 4.912464985994398e-05, 'epoch': 0.10504201680672269}
{'loss': 0.1991, 'grad_norm': 1.2549102306365967, 'learning_rate': 4.868697478991597e-05, 'epoch': 0.15756302521008403}
{'loss': 0.221, 'grad_norm': 1.2652634382247925, 'learning_rate': 4.824929971988796e-05, 'epoch': 0.21008403361344538}
{'loss': 0.1859, 'grad_norm': 3.0163862705230713, 'learning_rate': 4.782037815126051e-05, 'epoch': 0.26260504201680673}
{'loss': 0.1699, 'grad_norm': 6.456760883331299, 'learning_rate': 4.738270308123249e-05, 'epoch': 0.31512605042016806}
{'loss': 0.184, 'grad_norm': 1.4362233877182007, 'learning_rate': 4.6945028011204486e-05, 'epoch': 0.36764705882352944}
{'loss': 0.1671, 'grad_norm': 4.505099773406982, 'learning_rate': 4.6507352941176475e-05, 'epoch': 0.42016806722689076}
{'loss': 0.157,

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-7c9304cf28c1>:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training model for python...
{'loss': 0.5235, 'grad_norm': 3.22928786277771, 'learning_rate': 4.909090909090909e-05, 'epoch': 0.21231422505307856}
{'loss': 0.4678, 'grad_norm': 2.869610548019409, 'learning_rate': 4.812379110251451e-05, 'epoch': 0.42462845010615713}
{'loss': 0.4222, 'grad_norm': 4.54914665222168, 'learning_rate': 4.7156673114119924e-05, 'epoch': 0.6369426751592356}
{'loss': 0.381, 'grad_norm': 4.232608795166016, 'learning_rate': 4.6189555125725344e-05, 'epoch': 0.8492569002123143}
{'loss': 0.3427, 'grad_norm': 7.279270648956299, 'learning_rate': 4.522243713733076e-05, 'epoch': 1.0615711252653928}
{'loss': 0.2635, 'grad_norm': 10.65876579284668, 'learning_rate': 4.425531914893617e-05, 'epoch': 1.2738853503184713}
{'loss': 0.2517, 'grad_norm': 7.190958499908447, 'learning_rate': 4.328820116054158e-05, 'epoch': 1.48619957537155}
{'loss': 0.2485, 'grad_norm': 7.922757625579834, 'learning_rate': 4.2321083172147e-05, 'epoch': 1.6985138004246285}
{'loss': 0.2664, 'grad_norm': 

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-7c9304cf28c1>:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training model for pharo...
{'loss': 0.4483, 'grad_norm': 2.8776259422302246, 'learning_rate': 4.873971193415638e-05, 'epoch': 0.3076923076923077}
{'loss': 0.3446, 'grad_norm': 2.430668830871582, 'learning_rate': 4.745370370370371e-05, 'epoch': 0.6153846153846154}
{'loss': 0.3077, 'grad_norm': 14.67634391784668, 'learning_rate': 4.616769547325103e-05, 'epoch': 0.9230769230769231}
{'loss': 0.235, 'grad_norm': 3.7626655101776123, 'learning_rate': 4.4881687242798354e-05, 'epoch': 1.2307692307692308}
{'loss': 0.2089, 'grad_norm': 4.757662773132324, 'learning_rate': 4.359567901234568e-05, 'epoch': 1.5384615384615383}
{'loss': 0.2112, 'grad_norm': 6.66446590423584, 'learning_rate': 4.230967078189301e-05, 'epoch': 1.8461538461538463}
{'loss': 0.183, 'grad_norm': 2.1654770374298096, 'learning_rate': 4.104938271604938e-05, 'epoch': 2.1538461538461537}
{'loss': 0.1549, 'grad_norm': 2.193552017211914, 'learning_rate': 3.976337448559671e-05, 'epoch': 2.4615384615384617}
{'loss': 0.1247, 'grad_norm

In [ ]:
!pip install huggingface-hub
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import HfApi, login
from google.colab import userdata

hf_token = userdata.get('HF_ACCESS_TOKEN')

login(token=hf_token)

api = HfApi()

for lang in langs:
    repo_name = f"graphic-nlbse-{lang}"

    repo_url = api.create_repo(repo_id=repo_name, exist_ok=True)

    model_dir = f"./graphcodebert_models/{lang}_model"
    tokenizer_dir = f"./graphcodebert_models/{lang}_tokenizer"

    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

    model.push_to_hub(repo_id=repo_name)
    tokenizer.push_to_hub(repo_id=repo_name)

    print(f"Model and tokenizer for {lang} successfully pushed to repository: {repo_url}")

print("All models and tokenizers have been successfully uploaded.")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model and tokenizer for java successfully pushed to repository: https://huggingface.co/harisathar04/graphic-nlbse-java


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model and tokenizer for python successfully pushed to repository: https://huggingface.co/harisathar04/graphic-nlbse-python


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model and tokenizer for pharo successfully pushed to repository: https://huggingface.co/harisathar04/graphic-nlbse-pharo
All models and tokenizers have been successfully uploaded.


# **Evaluation**
The model's performance was assessed using standard classification metrics, including precision, recall, and F1 scores, to ensure effectiveness on the task.

In [ ]:
import time
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset

total_flops = 0
total_time = 0
scores = []

for lang in langs:
    model = AutoModelForSequenceClassification.from_pretrained(f'./graphcodebert_models/{lang}_model')
    tokenizer = AutoTokenizer.from_pretrained(f'./graphcodebert_models/{lang}_tokenizer')

    model.eval()
    if torch.cuda.is_available():
        model = model.to('cuda')

    test_dataset = ds[lang + '_test']
    inputs = tokenizer(test_dataset['combo'], truncation=True, padding=True, return_tensors="pt")

    dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
    batch_size = 32
    dataloader = DataLoader(dataset, batch_size=batch_size)

    total_outputs = []

    lang_flops = 0
    start_time = 0
    stop_time = 0
    total = 0

    with torch.no_grad():
        for input_batch in dataloader:
            input_ids, attention_mask = input_batch
            if torch.cuda.is_available():
                input_ids, attention_mask = input_ids.to('cuda'), attention_mask.to('cuda')

            with torch.profiler.profile(
                activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
                with_flops=True
            ) as profiler:
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            batch_flops = sum([e.flops for e in profiler.key_averages() if e.flops is not None]) / len(dataloader)
            lang_flops += batch_flops / 1e9

            logits = torch.sigmoid(outputs.logits)
            total_outputs.append(logits.cpu().numpy())

    total_flops += lang_flops
    start_time = time.time()
    for i in range(10):
        y_pred = (np.concatenate(total_outputs, axis=0) > 0.5).astype(int).T
    total_time += time.time() - start_time
    print(f"Language: {lang}, GFLOPs: {lang_flops:.2f}")

print("Total Compute in GFLOPs:", total_flops)
print("Average runtime in seconds:", total_time / 10)

Language: java, GFLOPs: 2778.66
Language: python, GFLOPs: 196.35
Language: pharo, GFLOPs: 270.07
Total Compute in GFLOPs: 3245.0810635754224
Average runtime in seconds: 0.00010399818420410156


In [ ]:
# Evaluation (same as before)
from torch.cuda.amp import autocast

scores = []

for lan in langs:
    num_labels = len(labels[lan])

    model = AutoModelForSequenceClassification.from_pretrained(f'./graphcodebert_models/{lan}_model')
    tokenizer = AutoTokenizer.from_pretrained(f'./graphcodebert_models/{lan}_tokenizer')

    model.eval()
    if torch.cuda.is_available():
        model = model.to('cuda')

    test_dataset = ds[lan + '_test']
    inputs = tokenizer(test_dataset['combo'], truncation=True, padding=True, return_tensors="pt")

    dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
    batch_size = 32
    dataloader = DataLoader(dataset, batch_size=batch_size)

    total_outputs = []

    with torch.no_grad():
        for input_batch in dataloader:
            input_ids, attention_mask = input_batch
            if torch.cuda.is_available():
                input_ids, attention_mask = input_ids.to('cuda'), attention_mask.to('cuda')

            with autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                total_outputs.append(outputs.logits.cpu().numpy())

    total_outputs = np.concatenate(total_outputs, axis=0)
    y_pred = (total_outputs > 0).astype(int).T
    y_true = np.array(test_dataset['labels']).T

    for i in range(len(y_pred)):
        assert(len(y_pred[i]) == len(y_true[i]))

        tp = sum([true == pred == 1 for (true, pred) in zip(y_true[i], y_pred[i])])
        tn = sum([true == pred == 0 for (true, pred) in zip(y_true[i], y_pred[i])])
        fp = sum([true == 0 and pred == 1 for (true, pred) in zip(y_true[i], y_pred[i])])
        fn = sum([true == 1 and pred == 0 for (true, pred) in zip(y_true[i], y_pred[i])])

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = (2 * tp) / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0

        scores.append({'lan': lan, 'cat': labels[lan][i], 'precision': precision, 'recall': recall, 'f1': f1})

scores_df = pd.DataFrame(scores)


avg_precision = scores_df['precision'].mean()
avg_recall = scores_df['recall'].mean()
avg_f1 = scores_df['f1'].mean()

print(f"Average Precision: {avg_precision}")
print(f"Average Recall: {avg_recall}")
print(f"Average F1 Score: {avg_f1}")
scores_df

<ipython-input-24-1abf7818d810>:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Average Precision: 0.7083832983582361
Average Recall: 0.7258663864050771
Average F1 Score: 0.7134510707861819


,lan,cat,precision,recall,f1
0,java,summary,0.891232,0.900224,0.895706
1,java,Ownership,1.000000,1.000000,1.000000
2,java,Expand,0.429907,0.450980,0.440191
3,java,usage,0.936869,0.860789,0.897219
4,java,Pointer,0.806306,0.972826,0.881773
5,java,deprecation,0.785714,0.733333,0.758621
6,java,rational,0.354430,0.411765,0.380952
7,python,Usage,0.841121,0.743802,0.789474
8,python,Parameters,0.846774,0.820312,0.833333
9,python,DevelopmentNotes,0.340909,0.365854,0.352941


# **Submission Score**

In [ ]:
max_avg_runtime = 5
max_avg_flops = 5000

def submission_score(avg_f1, avg_runtime, avg_flops):
    return (
        0.6 * avg_f1 +
        0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
        0.2 * ((max_avg_flops - avg_flops) / max_avg_flops)
    )

avg_runtime = total_time / 10
avg_flops = total_flops / 10

final_score = round(submission_score(avg_f1, avg_runtime, avg_flops), 2)
print(f"Submission Score: {final_score}")

Submission Score: 0.82
